In [ ]:
import numpy as np
import sympy as sp

In [ ]:
GRAVITY = 10.


class State(object):
    def __init__(self):
        pass
    
    def __repr__(self):
        return 'q<{:x}>'.format(hash(self))

    
class HingeRod(object):
    def __init__(self, length=2., mass=1.):
        self.theta = State()
        self.length = length
        self.mass = mass

    def get_pos(self, state_map):
        theta, _ = state_map[self.theta]
        return self.length * np.matrix([[np.sin(theta)], [-np.cos(theta)]])
    
    def get_vel(self, state_map):
        theta, dtheta = state_map[self.theta]
        return self.length * dtheta * np.matrix([[np.cos(theta)], [np.sin(theta)]])

    def get_accel_matrix(self, state_map):
        '''The acceleration matrix is meant to be multiplied with a column
        vector that consists of each of the states' second derivatives, as
        well as a final entry that corresponds to the value 1.
        
        This is represented as a matrix because we're actually trying to
        solve for the second derivatives of each of the states, as compared
        to getting an actual value.
        '''
        # Rows: x,y coordinates
        # Cols: states + [1]
        theta, dtheta = state_map[self.theta]
        return np.matrix([
            [
                self.length * np.cos(theta),
                -self.length * dtheta**2 * np.sin(theta),
            ],
            [
                self.length * np.sin(theta),
                self.length * dtheta**2 * np.cos(theta),
            ]
        ])
    
    def get_vel_perturbation(self, state_map, state):
        '''Measure the amount of perturbation to the velocity relative to
        a change in a state's momentum value.
        '''
        if state == self.theta:
            theta, _ = state_map[self.theta]
            return self.length * np.matrix([[np.cos(theta)], [np.sin(theta)]])
        else:
            return np.matrix([[0.], [0.]])
        
    def get_vel_pos_perturbation(self, state_map, state):  # god we need better names
        if state == self.theta:
            theta, dtheta = state_map[self.theta]
            return self.length * dtheta * np.matrix([[-np.sin(theta)], [np.cos(theta)]])
        else:
            return np.matrix([[0.], [0.]])        
    
    def get_vel_perturbation_deriv(self, state_map, state):
        '''Get the time-derivative of the vel_perturbation vector.'''
        return np.matrix([[0.], [0.]])
    
    def get_potential_perturbation(self, state_map, state):
        '''Measure the amount of perturbation to the potential energy
        relative to a change in a state's position value.
        '''
        if state == self.theta:
            theta, _ = state_map[self.theta]
            return self.length * np.sin(theta)
        else:
            return 0.
    

num_objs = 2
objs = [HingeRod() for _ in range(num_objs)]
states = [x.theta for x in objs]

state_map = {
    objs[0].theta: [
        1.8,
        0.,
    ],
    objs[1].theta: [
        0.4,
        0.,
    ],
}

def get_onehot_vector(index, length):
    a = np.zeros(length)
    a[index] = 1.
    return np.matrix(a).T


def get_spacer_matrix(num_states, state_num):
    m = np.matrix(np.zeros([2, num_states + 1]))
    m[0, state_num] = 1.
    m[1, num_states] = 1.
    return m


def get_K(objs, state_map, state, num_states, obj_num, state_num):
    #print('state_num: {}'.format(state_num))
    mass = objs[obj_num].mass
    padder = np.matrix(np.append(np.zeros(num_states), 1))
    result = np.matrix(np.zeros([1, num_states + 1]))
    for i in range(obj_num + 1):
        for j in range(obj_num + 1):
            spacer = get_spacer_matrix(num_states, j)
            m1 = mass * objs[i].get_vel_perturbation(state_map, state).T * objs[j].get_accel_matrix(state_map) * spacer
            m2 = mass * objs[i].get_vel_perturbation_deriv(state_map, state).T * objs[j].get_vel(state_map) * padder
            m3 = objs[i].get_vel_pos_perturbation(state_map, state).T * objs[j].get_vel(state_map) * padder
            result += m1
            result += m2
            result -= m3
        result -= mass * GRAVITY * objs[i].get_potential_perturbation(state_map, state) * padder
    
    #m = np.matrix(np.zeros(num_states, num_states + 1))
    return result
    

num_states = len(states)

def tick(state_map, dt=0.01):
    M = np.matrix(np.zeros([2, num_states + 1]))
    for obj_num in range(0, num_objs):
        for state_num, state in enumerate(states):
            expander = get_onehot_vector(state_num, num_states)
            more = expander * get_K(objs, state_map, state, num_states, obj_num, state_num)
            M += more

    new_state_map = {}
    crunched = sp.Matrix(M).rref()
    for state_num, state in enumerate(states):
        q, dq = state_map[state]
        ddq = -float(crunched[0][state_num, -1])
        new_state_map[state] = [q + dq*dt, dq + ddq*dt]
    return new_state_map

state_maps = [state_map]
for i in range(100):
    state_map = tick(state_map)
    state_maps.append(state_map)

    
np.matrix([x[objs[0].theta] + x[objs[1].theta] for x in state_maps])
#np.matrix([x[objs[0].theta] for x in state_maps])
